**cross validation 한다**

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


**데이터 전처리는 동일**

In [0]:
import pandas as pd
import numpy as np

data = pd.read_csv("/gdrive/My Drive/classification/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df_sample = data.loc[:, ['Init_Win_bytes_forward', ' Avg Fwd Segment Size',' Init_Win_bytes_backward', ' Fwd Header Length.1',' Label']]
df_labels= df_sample[' Label']
df_packets=df_sample.drop([' Label'],axis='columns')

#같은 길이 나오는 거확인
print(len(df_sample))
print(len(df_labels))
print(len(df_packets))

#콜롬 목록과 데이터 타입 확인 후 변환
df_packets.columns
#type(df_sample)
df_packets = df_packets.astype(float) #형변환

#리스트로 되돌리기
packets= df_packets.values.tolist()
labels= df_labels.values.tolist()


#확인
packets[1]
labels[1]

#길이 확인
len(packets)
len(labels)
#len(packets[1]) #한개당 80차원

#기왕 하는거 labels도 float으로 바꿔주기
for c in range(225745):
  if (labels[c] == 'BENIGN' ):
    labels[c]=0 #false
  else:
    labels[c] =1 #true
 

#테스트 군과 학습군으로 나누기.
#전체는 225745개. (70%는 약 158,021 개)
train_set = packets[:158021]
test_set = packets[158021:]
train_label =labels[:158021]
test_label = labels[158021:]

train_set = np.asarray(train_set)
train_label = np.asarray(train_label)
test_set = np.asarray(test_set)
test_label = np.asarray(test_label)


#잘 나눠졌는지 확인
print(len(train_set) + len(test_set))
print(len(test_label) + len(train_label))

train_set = np.expand_dims(train_set, axis=2) #reshape
test_set = np.expand_dims(test_set, axis=2) #reshape


225745
225745
225745
225745
225745


In [0]:
#cross validation
import tensorflow as tf
from sklearn.model_selection import KFold
num_folds = 10 #폴드 수 결정

kfold = KFold(n_splits=num_folds, shuffle=True)


# acc와 lose를 저장해뒀다가 나중에 평균치 계산
acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
inputs = np.concatenate((test_set, train_set), axis=0)
targets = np.concatenate((test_label, train_label), axis=0)


#모델
foldnum = 1
for train, test in kfold.split(inputs, targets):
  model = tf.keras.Sequential([
  tf.keras.layers.Conv1D(16, (2), activation='relu', input_shape=(4, 1)),
  tf.keras.layers.MaxPooling1D(2,padding='same'),
  tf.keras.layers.Conv1D(16, (2), activation='relu'),
  tf.keras.layers.MaxPooling1D(2, padding='same'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(15, activation='softmax')
  ])
  #compile
  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  #결과
  history = model.fit(inputs[train], targets[train], epochs=5) #10번의 epoch
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {foldnum}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])
  #다음 fold로
  foldnum = foldnum + 1


print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')


Epoch 1/5
6350/6350 [==============================] - 20s 3ms/step - loss: 0.5534 - accuracy: 0.9632
Epoch 2/5
6350/6350 [==============================] - 20s 3ms/step - loss: 0.0663 - accuracy: 0.9851
Epoch 3/5
6350/6350 [==============================] - 20s 3ms/step - loss: 0.0478 - accuracy: 0.9888
Epoch 4/5
6350/6350 [==============================] - 20s 3ms/step - loss: 0.0581 - accuracy: 0.9875
Epoch 5/5
6350/6350 [==============================] - 21s 3ms/step - loss: 0.0264 - accuracy: 0.9942
Score for fold 1: loss of 0.029964419081807137; accuracy of 99.43743348121643%
Epoch 1/5
6350/6350 [==============================] - 20s 3ms/step - loss: 0.5233 - accuracy: 0.9605
Epoch 2/5
6350/6350 [==============================] - 20s 3ms/step - loss: 0.0688 - accuracy: 0.9844
Epoch 3/5
6350/6350 [==============================] - 20s 3ms/step - loss: 0.1281 - accuracy: 0.9581
Epoch 4/5
6350/6350 [==============================] - 20s 3ms/step - loss: 0.1617 - accuracy: 0.9460
Epo